In [1]:
import os
os.chdir('C:/Users/RH/A_whole_try_prog/y_test_data')

In [2]:
import pandas as pd
import numpy as np
import esm
import torch

print(pd.__version__)
print(np.__version__)
print(esm.__version__)
print(torch.__version__)

1.5.3
1.22.0
2.0.0
2.0.0


In [3]:
import numpy as np
import pandas as pd

In [4]:
#training dataset loading
#打开指定路径的excel文件
excel_path = r'C:\Users\RH\A_whole_try_prog\X_test_nr.xlsx'
dataset = pd.read_excel(excel_path, na_filter=False)
sequence_list = dataset['Sequence']

In [5]:
print(sequence_list)

0                  AHK
1                  VHH
2              VHHANEN
3             VNPHDHQN
4            LVNPHDHQN
5           LLPHHADADY
6              VIPAGYP
7     LQSGDALRVPSGTTYY
8                  VKL
9                 VVKL
10                 IVF
11                 FSL
12                  PP
13              SVPQPK
14               EVPKA
15              PLAQPA
16               LLNPT
17               HLPLP
18              VLPIPQ
19                 LWL
Name: Sequence, dtype: object


In [6]:
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence)
print(peptide_sequence_list)

[('AHK', 'AHK'), ('VHH', 'VHH'), ('VHHANEN', 'VHHANEN'), ('VNPHDHQN', 'VNPHDHQN'), ('LVNPHDHQN', 'LVNPHDHQN'), ('LLPHHADADY', 'LLPHHADADY'), ('VIPAGYP', 'VIPAGYP'), ('LQSGDALRVPSGTTYY', 'LQSGDALRVPSGTTYY'), ('VKL', 'VKL'), ('VVKL', 'VVKL'), ('IVF', 'IVF'), ('FSL', 'FSL'), ('PP', 'PP'), ('SVPQPK', 'SVPQPK'), ('EVPKA', 'EVPKA'), ('PLAQPA', 'PLAQPA'), ('LLNPT', 'LLNPT'), ('HLPLP', 'HLPLP'), ('VLPIPQ', 'VLPIPQ'), ('LWL', 'LWL')]


In [7]:
#调试完毕，以此为准！
import torch
import esm
import pandas as pd

#load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()
    
#load data
data = peptide_sequence_list
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    
# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=True)
token_representations = results["representations"][33]
    
# Generate per-sequence representations via averaging
sequence_representations = []
for i, token_len in enumerate(batch_lens):
    each_seq_rep = token_representations[i, 1:token_len - 1].mean(0).tolist()  
    sequence_representations.append(each_seq_rep)

embedding_results = pd.DataFrame(sequence_representations)
print(embedding_results)
embedding_results.to_csv('X_test_1280.csv')

        0         1         2         3         4         5         6     \
0   0.036479  0.132322 -0.012477  0.172230 -0.057646 -0.102762 -0.221309   
1   0.089368  0.029520  0.016745  0.001059 -0.128657 -0.128163 -0.256488   
2   0.014724  0.121371  0.062733  0.100393 -0.032653 -0.147418 -0.151980   
3  -0.005682  0.094091  0.037555  0.104655  0.049339 -0.196482 -0.148488   
4   0.024735  0.089302  0.046541  0.080209  0.032991 -0.205648 -0.150509   
5  -0.014631  0.155684  0.005371  0.080860  0.055000 -0.083015 -0.142902   
6  -0.024842  0.087603 -0.018586  0.062196  0.018029 -0.129932 -0.100574   
7  -0.040522  0.070905  0.091476  0.106613 -0.054882 -0.068831 -0.049461   
8   0.030423  0.055366  0.093223  0.038960 -0.055797 -0.063598 -0.187947   
9   0.000249  0.065227  0.095760  0.041539 -0.066741 -0.066292 -0.197467   
10  0.079011  0.098246  0.176288  0.069504 -0.093190 -0.084974 -0.142306   
11  0.111859  0.060217  0.290169  0.089864 -0.026091 -0.061514 -0.205418   
12  0.010950